In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import
import pandas as pd 
import numpy as np
import os
import librosa

from pydub import AudioSegment
from pydub.playback import play
from pydub.utils import make_chunks

import matplotlib.pyplot as plt
import librosa.display

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, train_test_split, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier

# Function Explanation
(name of func.): explain

- convert_data(file_path, before_format, after_format): convert original file`s format
- segment_data(main_path, file_path, save_path, sec): The data is  segmented by sec



In [ ]:
# Convert file_type 
def convert_data(file_path, before_format, after_format):

  '''
  [Explain of variables]
  path: the data location
  before_format: original file format
  after_format: file format after converting
  '''

  # pathDataSet = "/content/drive/Shareddrives/AI-Drone/Dataset/0516 Indoor"

  for idx, f in enumerate(os.listdir(file_path)):
    try:

      # if the file is not directory, then exec
      if os.path.isfile(os.path.join(file_path, f)):
        # load audio data 
        track = AudioSegment.from_file(os.path.join(file_path, f), format=before_format) # mp4
        # save converted audio data
        new_f = track.export(os.path.join(file_path, "wav", f[:-4]+'.wav'), format=after_format) # wav
        print("Success %d - %s"%(idx, f)) 
        
    except Exception as e: 
      print(e)
      print("Fail %d - %s"%(idx, f)) 
      break

convert_data("/content/drive/Shareddrives/AI-Drone/Dataset/0516 Indoor", "mp4", "wav")

Success 0 - NoDrone.mp4
Success 1 - 3m_SmallDrone_2.mp4
Success 2 - 3m_SmallDrone_1.mp4
Success 3 - 2m_SmallDrone_1.mp4
Success 4 - 2m_BigDrone_1.mp4
Success 5 - 1m_SmallDrone_1.mp4
Success 6 - 1m_BigDrone_1.mp4
Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librub

In [ ]:
# Segment wav file
def segment_data(main_path, file_path, save_path, sec):

  '''
  [Explain of variables]
  main_path: the root location
  file_path: the data location
  save_path: the data location for saving
  sec: second that user wanna segment (ms)
  '''

  # pathDataSet = "/content/drive/Shareddrives/AI-Drone/Dataset/0516 Indoor"
  # pathWave = "wav"
  # pathSegment = "segmented"

  for idx, f in enumerate(os.listdir(os.path.join(main_path, file_path))):
    try:
      
      # Just wav file
      if f[-3:] == 'wav':
        print("<<< ", f, " >>>")
        # load the wave file
        audio = AudioSegment.from_file(os.path.join(main_path, file_path, f))

        # Separate by 3s
        chunk_legth_ms = sec # 3000ms
        chunks = make_chunks(audio, chunk_legth_ms)

        # Save each .wav
        for i, chunk in enumerate(chunks):
          # if the each duration same with chunk length
          if int(chunk.duration_seconds) == chunk_legth_ms//1000:
            chunk_name = f[:-4] + "-{0}.wav".format(i)
            chunk.export(os.path.join(main_path, save_path, chunk_name), format="wav")

    except Exception as e: 
      print(e)
      print("Fail to segment >>> ", f)
      break

segment_data("/content/drive/Shareddrives/AI-Drone/Dataset/0516 Indoor", "wav", "segmented", 3000)

<<<  NoDrone.wav  >>>
<<<  3m_SmallDrone_2.wav  >>>
<<<  3m_SmallDrone_1.wav  >>>
<<<  2m_SmallDrone_1.wav  >>>
<<<  2m_BigDrone_1.wav  >>>
<<<  1m_SmallDrone_1.wav  >>>
<<<  1m_BigDrone_1.wav  >>>


In [ ]:
def save_as_image(mfccs, f):
  pathDataSet = "/content/drive/Shareddrives/AI-Drone/Dataset/0516 Indoor"
  pathImage = "mfcc(image)"

  # Save as image (.jpg)
  plt.figure(figsize=(10, 4))
  librosa.display.specshow(mfccs)
  plt.savefig(os.path.join(pathDataSet, pathImage, f[:-4]+".jpg"))


In [ ]:
def save_as_csv(mfccs, file_name):
  pathDataSet = "/content/drive/Shareddrives/AI-Drone/Dataset/0516 Indoor"
  pathCsv = "mfcc(value)"

  # Save as float  
  trans_mfccs = mfccs.T # Transpose (feature, time) to (time, feature) 
  pd.DataFrame(trans_mfccs).to_csv(os.path.join(pathDataSet, pathCsv, file_name[:-4] + ".csv"))


In [ ]:
# Extract Feature using MFCC
pathDataSet = "/content/drive/Shareddrives/AI-Drone/Dataset/0516 Indoor"
pathSegmented = "segmented"

for idx, f in enumerate(os.listdir(os.path.join(pathDataSet, pathSegmented))):
  try:
    
    # wav file
    if f[-3:] == 'wav':
      # Extract MFCC`s feature
      y, sr = librosa.load(os.path.join(pathDataSet, pathSegmented, f))
      # librosa.display.waveplot(y, sr=sr)
      mfccs = librosa.feature.mfcc(y=y, sr=sr)
      
      # Call func.
      save_as_csv(mfccs, f)
      save_as_image(mfccs, f)

      print("<<< %d - %s >>>"%(idx, f))

  except:
    print("Fail to segment >>> ", f)
    break

Class: Small, Large, None

In [ ]:
# Counting the number of dataset for each day/class
pathDataSet = "/content/drive/Shareddrives/AI-Drone/Dataset"
pathList = ["0516 Indoor", "0517 Indoor", "0518 Indoor", "0519 Indoor"]

pathValue = "mfcc(value)"
pathImage = "mfcc(image)"
small, large, none = 0, 0, 0
for pl in pathList:
  for idx, f in enumerate(os.listdir(os.path.join(pathDataSet, pl, pathValue))):
    if "Small" in f:
      small += 1
    elif "Big" in f:
      large += 1
    elif "No" in f:
      none += 1
print(small, large, none)
print("Total: ", small + large + none)

429 377 1068
Total:  1874


Putting MFCC image files into a different directory

In [ ]:
import shutil

In [ ]:
main_src_path = "/content/drive/Shareddrives/AI-Drone/Dataset"
src_path = ["0516 Indoor", "0517 Indoor", "0518 Indoor", "0519 Indoor",]
file_path = "mfcc(image)"

main_dst_path = "/content/drive/Shareddrives/AI-Drone/Dataset/Audio(MFCC Image)/juann"
dst_path = ["Drone", "NoDrone"]

In [ ]:
drone, noDrone = 0, 0

for src in src_path:
  for f in os.listdir(os.path.join(main_src_path, src, file_path)):
    if "No" in f:
      shutil.move(os.path.join(main_src_path, src, file_path, f), 
                  os.path.join(main_dst_path, dst_path[1], "NoDrone_"+str(noDrone)))
      noDrone += 1

    elif "Big" in f or "Small" in f:
      shutil.move(os.path.join(main_src_path, src, file_path, f), 
                  os.path.join(main_dst_path, dst_path[0], "Drone_"+str(drone)))
      drone += 1
    print("Successd >> ", f)